In [380]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [381]:
agent1['url'] = "http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMDI4OWVlYTktMDU4Ny00ZTc4LTg2ZGEtODIyZDMyODFiY2ZiIiwgInJlY2lwaWVudEtleXMiOiBbIjEzYlR1RDYzcGJhSmZoMm16cEpjU3NQdXl0QThTc1A0ajd3ZHRGVmRrMXBoIl0sICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QifQ=="

In [382]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '0289eea9-0587-4e78-86da-822d3281bcfb',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['13bTuD63pbaJfh2mzpJcSsPuytA8SsP4j7wdtFVdk1ph'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [383]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2PdYpQB79GBH7K5gH9zWFmt313uYeGxqrsqCRXWyDs1TqSNAud2mz4JNduZeSKeXeKjzZxC9KM5Gada53wTQNjRG

Public key: 
 DEBMWAgADpW1JRcnhkBidDaB64ifFdRoRD56ivDj7tJA


**Sending a connection request to acapy**

In [384]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [385]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [386]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [387]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "56ea79f5-0943-40e5-887d-610886a84388", "~thread": {"thid": '
 '"8aea75cd-813a-40a2-bab9-008d65578079"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"C9Dloc9a1shLtl6GVgUrt0E3HpaaGHZ7WZOMnOotQon1H08keVzxYm94NbHiVDOH6Gfnyix2fUvWlSpAod5iAw==", '
 '"sig_data": '
 '"AAAAAF69CcR7IkRJRCI6ICJUdkRYV1dHYmYyRjQ3ZTV0a2FhWDRkIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpUdkRYV1dHYmYyRjQ3ZTV0a2FhWDRkIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6VHZEWFdXR2JmMkY0N2U1dGthYVg0ZCMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OlR2RFhXV0diZjJGNDdlNXRrYWFYNGQiLCAicHVibGljS2V5QmFzZTU4IjogIkZmdjdkeHpXWUFKZzlodzF4Yzg4M0dpQlltajJpYkY4dzN4cW5yVnNBSGgzIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [388]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'TvDXWWGbf2F47e5tkaaX4d',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:TvDXWWGbf2F47e5tkaaX4d#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d',
            'publicKey': [{'controller': 'did:sov:TvDXWWGbf2F47e5tkaaX4d',
                           'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d#1',
                           'publicKeyBase58': 'Ffv7dxzWYAJg9hw1xc883GiBYmj2ibF8w3xqnrVsAHh3',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d;indy',
                         'priority': 0,
                         'recipientKeys': ['Ffv7dxzWYAJg9hw1xc883GiBYmj2ibF8w3xqnrVsAHh3'],
                         'serviceEndpoint': 'http://agent2.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [389]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:TvDXWWGbf2F47e5tkaaX4d#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d',
             'publicKey': [{'controller': 'did:sov:TvDXWWGbf2F47e5tkaaX4d',
                            'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d#1',
                            'publicKeyBase58': 'Ffv7dxzWYAJg9hw1xc883GiBYmj2ibF8w3xqnrVsAHh3',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:TvDXWWGbf2F47e5tkaaX4d;indy',
                          'priority': 0,
                          'recipientKeys': ['Ffv7dxzWYAJg9hw1xc883GiBYmj2ibF8w3xqnrVsAHh3'],
                          'serviceEndpoint': 'http://agent2.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Client (admin)',
  'myKey': b'E\x88I\xfa\xc9\x98\x93\x14lz_\xbf\x87\xfd\xc6\xf5o\x82

### FEATURE DISCOVERY

In [390]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '7c0afca4-59f2-4628-9cf8-c47b23064336',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'https://didcomm.org/issue-credential/1.0'},
               {'pid': 'https://didcomm.org/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/credential-presentation/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'https://didcomm.org/present-proof/1.0'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/credential-issuance/0.1'},
      

## Dids

In [391]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '6392c9c3-c3f6-4651-a68a-afd1575e467a',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0/problem-report',
 'explain-ltxt': 'Unrecognized message type '
                 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids',
 '~thread': {'thid': '8f486a91-f799-4ff1-8864-5f65def969d4'}}


## Public did

In [392]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'eb511a10-302d-4b8d-8e7c-4dcac4e7f5b5',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0/problem-report',
 'explain-ltxt': 'Unrecognized message type '
                 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did',
 '~thread': {'thid': '871d5b05-7f6e-455a-b062-7f5d656c5b81'}}


In [393]:

message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'd7b854ee-44f4-4309-8855-a347524905d1',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/disclose',
 'protocols': 'hello world',
 '~thread': {'thid': 'ef6e3f37-0926-4165-91eb-9ac2d5e043ce'}}
